In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [25]:
from pathlib import Path
from matplotlib import pyplot as plt
import cv2
import tqdm
from ipywidgets import widgets
from IPython.display import clear_output
import rasterio
import functools
import numpy as np
from PIL import Image

import utils
from sampler import BackgroundSampler
from lightning import cut_off_light_from_img
from rle2tiff import save_mask

import functools
import numpy as np

#### Configurations

In [29]:
path_to_img_folder = Path('data/tiff/')
path_to_mask_folder = Path('data/mask/')
path_to_anot_struct_folder = Path('data/markup/')
postfix = '-anatomical-structure.json'
out_path_to_light_mask = Path('data/lightning_mask/')
out_path_to_light_img = Path('data/lightning_img/')

num_samples = 10 # Total number of lightning templates
wh = (512, 512)

#### Create background samples 

Choose only images with lightnings

In [54]:
path_to_imgs = sorted(list(path_to_img_folder.glob('*')))
indices_img_with_light = [2]  # Chosen manually
path_to_imgs = [path_to_imgs[ind] for ind in indices_img_with_light]

In [60]:
out_path_to_light_mask.mkdir(exist_ok=True) 
out_path_to_light_img.mkdir(exist_ok=True) 

for path_to_img in tqdm.tqdm(path_to_imgs):
    path_to_mask =path_to_mask_folder / path_to_img.name
    path_to_anot_struct = path_to_anot_struct_folder / (path_to_img.stem + postfix)
    num_samples_per_img = num_samples//len(path_to_imgs)
    
    # List of polygons containing anotomical structure
    polygons = utils.get_cortex_polygons(utils.jread(path_to_anot_struct))
    back_samples = BackgroundSampler(path_to_img, 
                                     path_to_mask, 
                                     polygons, 
                                     wh, 
                                     num_samples_per_img)
    for ind, back_sample in enumerate(back_samples):
        img, _ = back_sample
        mask_with_light = cut_off_light_from_img(img)
        if mask_with_light is not None:
            save_mask(mask_with_light, out_path_to_light_mask / f'{ind}_{path_to_img.name}')
            im = Image.fromarray(img[0])
            im.save(out_path_to_light_img / f'{ind}_{path_to_img.name}')

100%|██████████| 1/1 [01:03<00:00, 63.91s/it]


### Clean

In [61]:
samples = out_path_to_light_mask.glob('*')
plt.rcParams["figure.figsize"] = (12,6)

del_mem_path = []

def delete(light_mask, light_sample, nothing):
    del_mem_path.append(light_mask)
    del_mem_path.append(light_sample)
    
def f(nothing):
    clear_output(wait=True)
    display(button)
    
    try:
        light_mask = next(samples)
        img_name = light_mask.name
        light_sample = out_path_to_light_img / img_name

        mask = rasterio.open(light_mask)
        sample = Image.open(light_sample)

        fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.imshow(mask.read()[0], cmap='gray')
        ax2.imshow(sample, cmap='gray')
        plt.show()

        mask.close()
        sample.close()

        display(button_del)
        button_del.on_click(functools.partial(delete, str(light_mask), str(light_sample)))
    except StopIteration:
        pass

    
button = widgets.Button(description="Next")
button_del = widgets.Button(description="Delete")

display(button)
button.on_click(f)

Button(description='Next', style=ButtonStyle())